In [1]:
def count_para(model):
    trainable_parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    n_trainable_parameters = sum([p.numel() for p in trainable_parameters])
    return n_trainable_parameters

In [2]:
import os
os.chdir('/home/students/wittmann/jw-dir/repos/conditional_INNs/colorization_minimal_example')

In [3]:
import FrEIA
from model import *
cinn = ColorizationCINN(1e-3)

In [4]:
print(count_para(cinn.cinn), 'inn parameters')
print(count_para(cinn.cond_net), 'cond parameters')
print(count_para(cinn), 'total parameters')


print(list(cinn.modules()).__len__(), 'modules')
print(list(x for x in cinn.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock)).__len__(), 'coupling layers')
# 2 glow 2x64x64
# haar   8x32x32
# 4 glow
# split (1)    2x32x32                       | (2) 6x32x32 flat 6144
# haar       8x16x16
# 4 glow
# split (3)    4x16x16                       | (4) 4x16x16 flat 1024
# 4 glow
# flat 1024
# cat (2, 4, 3)

# print(count_para(list(x for x in cinn.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))[-1]), 'parameters fully')

print(list(count_para(x) for x in list(x for x in cinn.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))))

# 2x conv 3x3 32 (c64x64x64)
# 2x conv 1x1 64, conv 3x3 64 (c128x32x32)
# 2x conv 1x1 128, conv 3x3 128 (c128x16x16)
# 4x lin 1024 (c512)


# print(count_para(cinn.cinn), 'parameters')
print((cinn.cinn.dims_in), 'input')
print((cinn.cinn.dims_c), 'cond')


9555080 inn parameters
1455680 cond parameters
11010760 total parameters
168 modules
14 coupling layers
[38660, 38660, 18064, 161424, 18064, 161424, 36112, 322832, 36112, 322832, 2100224, 2100224, 2100224, 2100224]
[(2, 64, 64)] input
[(64, 64, 64), (128, 32, 32), (128, 16, 16), (512,)] cond


In [5]:
cinn.cinn(torch.rand(7,2,64,64), [torch.rand(7, 64, 64, 64), torch.rand(7, 128, 32, 32), torch.rand(7, 128, 16, 16), torch.rand(7, 512,)])[0].shape

torch.Size([7, 8192])

In [6]:
cinn(torch.rand(7,3,64,64))[0].shape

torch.Size([7, 8192])

In [7]:
for x in cinn.cinn.node_list_fwd:
    print(x, x.conditions, x.condition_dims)

ConditionNode '4fe1e0': [] ->  -> [(64, 64, 64)] [] []
InputNode '23b7a0': [] ->  -> [(2, 64, 64)] [] []
Node '477ad0': [(2, 64, 64)] -> GLOWCouplingBlock -> [(2, 64, 64)] [(ConditionNode '4fe1e0', 0)] [(64, 64, 64)]
Node 'bc0650': [(2, 64, 64)] -> GLOWCouplingBlock -> [(2, 64, 64)] [(ConditionNode '4fe1e0', 0)] [(64, 64, 64)]
ConditionNode '280e90': [] ->  -> [(128, 32, 32)] [] []
Node 'dba480': [(2, 64, 64)] -> HaarDownsampling -> ((8, 32, 32),) [] []
Node '477ce0': [(8, 32, 32)] -> GLOWCouplingBlock -> [(8, 32, 32)] [(ConditionNode '280e90', 0)] [(128, 32, 32)]
Node 'b02c30': [(8, 32, 32)] -> PermuteRandom -> [(8, 32, 32)] [] []
Node '4ac0e0': [(8, 32, 32)] -> GLOWCouplingBlock -> [(8, 32, 32)] [(ConditionNode '280e90', 0)] [(128, 32, 32)]
Node 'bfa9f0': [(8, 32, 32)] -> PermuteRandom -> [(8, 32, 32)] [] []
Node '4ac560': [(8, 32, 32)] -> GLOWCouplingBlock -> [(8, 32, 32)] [(ConditionNode '280e90', 0)] [(128, 32, 32)]
Node '4ac530': [(8, 32, 32)] -> PermuteRandom -> [(8, 32, 32)] []

In [8]:
cinn

ColorizationCINN(
  (cinn): GraphINN(
    (module_list): ModuleList(
      (0-1): 2 x GLOWCouplingBlock(
        (subnet1): Sequential(
          (0): Conv2d(65, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
          (2): Conv2d(32, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (subnet2): Sequential(
          (0): Conv2d(65, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
          (2): Conv2d(32, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (2): HaarDownsampling()
      (3): GLOWCouplingBlock(
        (subnet1): Sequential(
          (0): Conv2d(132, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU()
          (2): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
        )
        (subnet2): Sequential(
          (0): Conv2d(132, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU()
          (2): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
   